<a href="https://colab.research.google.com/github/benliao/gameai_notebook/blob/main/rllib_p1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Dependencies
======================

In [1]:
!pip install ray[debug]

# A hack to force the runtime to restart, need to include the above dependencies.
import os

os._exit(0)

     |████████████████████████████████| 49.4MB 62kB/s 
     |████████████████████████████████| 133kB 50.5MB/s 
     |████████████████████████████████| 3.1MB 30.9MB/s 
     |████████████████████████████████| 1.3MB 31.2MB/s 
     |████████████████████████████████| 10.1MB 33.0MB/s 
     |████████████████████████████████| 1.0MB 30.4MB/s 
     |████████████████████████████████| 81kB 9.3MB/s 
     |████████████████████████████████| 81kB 9.2MB/s 
     |████████████████████████████████| 71kB 7.9MB/s 
     |████████████████████████████████| 296kB 44.9MB/s 
     |████████████████████████████████| 143kB 44.6MB/s 
     |████████████████████████████████| 92kB 9.6MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-cp37-none-any.whl size=12621 sha256=204d0fcc3592c5a20236b6c4f346bdc621de2d3e5f9c04a535158928362d686a
  Stored in directory: /root/.cache/pip/wheels/48/b4/d5/fb5b7f1d040f2ff20687e3bad6867d63155dbde5a7c10f4293
Successfully built gpustat
  Found existing installation: protobuf 3.12.4
  

NameError: ignored

In [1]:
import gym
import numpy as np

In [2]:
env = gym.make("CartPole-v0")
print("Env:", env)

Env: <TimeLimit<CartPoleEnv<CartPole-v0>>>


In [3]:
state = env.reset()
print("State:", state)

State: [ 0.02312132 -0.0115145   0.04005199  0.00783099]


In [4]:
# Simulate taking an action in the environment. Appropriate actions for
# the CartPole environment are 0 and 1 (for moving left and right).
action = 0
state, reward, done, info = env.step(action)
print(state, reward, done, info)

[ 0.02289103 -0.20718728  0.04020861  0.31287692] 1.0 False {}


In [5]:
!pip install -U ray[rlib]

Requirement already up-to-date: ray[rlib] in /usr/local/lib/python3.7/dist-packages (1.4.0)


In [6]:
import gym
import ray
from ray.rllib.agents.ppo import PPOTrainer, DEFAULT_CONFIG
from ray.tune.logger import pretty_print

In [7]:
# Start up Ray. This must be done before we instantiate any RL agents.
ray.init(num_cpus=3, ignore_reinit_error=True, log_to_driver=False)

2021-06-18 06:52:11,742	INFO services.py:1274 -- View the Ray dashboard at http://127.0.0.1:8265


{'metrics_export_port': 62301,
 'node_id': '12d096ec9b8b1603ced059e08796551edfb1f48b70cc30c6bab90f66',
 'node_ip_address': '172.28.0.2',
 'object_store_address': '/tmp/ray/session_2021-06-18_06-52-09_928801_137/sockets/plasma_store',
 'raylet_ip_address': '172.28.0.2',
 'raylet_socket_name': '/tmp/ray/session_2021-06-18_06-52-09_928801_137/sockets/raylet',
 'redis_address': '172.28.0.2:6379',
 'session_dir': '/tmp/ray/session_2021-06-18_06-52-09_928801_137',
 'webui_url': '127.0.0.1:8265'}

In [8]:
config = DEFAULT_CONFIG.copy()
config['num_workers'] = 1
config['num_sgd_iter'] = 30
config['sgd_minibatch_size'] = 128
config['model']['fcnet_hiddens'] = [100, 100]
config['num_cpus_per_worker'] = 0  # This avoids running out of resources in the notebook environment when this cell is re-executed

agent = PPOTrainer(config, 'CartPole-v0')

2021-06-18 06:53:08,493	INFO logger.py:180 -- pip install 'ray[tune]' to see TensorBoard files.
2021-06-18 06:53:08,496	WARNING logger.py:317 -- Could not instantiate TBXLogger: No module named 'tensorboardX'.
2021-06-18 06:53:08,507	INFO trainer.py:671 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2021-06-18 06:53:08,511	INFO trainer.py:698 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2021-06-18 06:53:17,732	WARNING util.py:53 -- Install gputil for GPU system monitoring.


In [9]:
for i in range(2):
    result = agent.train()
    print(pretty_print(result))

agent_timesteps_total: 4000
custom_metrics: {}
date: 2021-06-18_06-54-07
done: false
episode_len_mean: 21.945054945054945
episode_media: {}
episode_reward_max: 79.0
episode_reward_mean: 21.945054945054945
episode_reward_min: 8.0
episodes_this_iter: 182
episodes_total: 182
experiment_id: 394b29ff70b94420a2599e3f7798efbc
hostname: a21c2114b0a3
info:
  learner:
    default_policy:
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 4.999999873689376e-05
        entropy: 0.6656601428985596
        entropy_coeff: 0.0
        kl: 0.028577415272593498
        model: {}
        policy_loss: -0.03588251769542694
        total_loss: 158.71192932128906
        vf_explained_var: 0.031721796840429306
        vf_loss: 158.74209594726562
  num_agent_steps_sampled: 4000
  num_agent_steps_trained: 4000
  num_steps_sampled: 4000
  num_steps_trained: 4000
iterations_since_restore: 1
node_ip: 172.28.0.2
num_healthy_workers: 1
off_policy_estimator: {}
perf:
  cpu_util_percent: 16